In [153]:
from utils import *
import random
import csv
import time
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from math import sqrt

In [154]:
def compute_query_score(query,utility_matrix,queries,usersID,k):
    queriesIDs=queries['queryID'].values.tolist()
    queries=queries['query'].values.tolist()
    #get a set of queries
    query_list=[]
    for i in queries:
        query={}
        for x in i.split(','):
            aux=x.split('=')
            query.update({aux[0]:aux[1]})
        query_list.append(query)
    #compute query similarity 
    values_similarity=[]
    for i in query_list:
        values_similarity.append(jaccard_similarity(query.values(),i.values()))
    values_similarity=np.array(values_similarity)
    queriesIDs=np.array(queriesIDs)
    #test if the query is already scored
    if values_similarity.max()==1:
        return utility_matrix[queriesIDs[values_similarity.argmax()]]
    else:
        #get the k nearest neighbor
        values_similarity,queriesIDs=KNN(k,values_similarity,queriesIDs)
        #compute scores
        scores=(np.array(utility_matrix[queriesIDs])@values_similarity)/values_similarity.sum()
    return pd.DataFrame(scores, index=usersID,columns=['Q'])

In [155]:
um= pd.read_csv('csv\\predicted_utility_matrix.csv',index_col=0)
users = pd.read_csv("./csv/users.csv",index_col=False)
usersID=users['User_Id'].values.tolist()
dataset=pd.read_csv("./csv/dataset.csv",index_col=False)


In [156]:
def transfrom_query(query):
    output={}
    for x in query.split(','):
            aux=x.split('=')
            output.update({aux[0]:aux[1]})
    return output

In [157]:
queries=pd.read_csv("./csv/queries.csv",index_col=0)
queriesIDs=um.columns.to_list()
rmse=0
for Id in queriesIDs:
    query=transfrom_query(queries['query'].iloc[queriesIDs.index(Id)])
    data=um.loc[:, ~um.columns.isin([Id])]
    df=compute_query_score(query,data,queries.loc[data.columns.to_list()].reset_index(),usersID,400)
    print(Id,round(sqrt(mean_squared_error(np.array(df),np.array(um[Id]))),2))
    rmse+=sqrt(mean_squared_error(np.array(df),np.array(um[Id])))
rmse=rmse/2000

Q1 50.57
Q2 50.5
Q3 26.39
Q4 8.13
Q5 25.86
Q6 26.41
Q7 45.7
Q8 13.43
Q9 62.23
Q10 45.39
Q11 51.05
Q12 50.84
Q13 34.24
Q14 26.28
Q15 35.1
Q16 58.85
Q17 26.51
Q18 61.76
Q19 8.25
Q20 8.33
Q21 34.17
Q22 8.22
Q23 45.52
Q24 8.23
Q25 34.65
Q26 45.38
Q27 45.16
Q28 45.73
Q29 50.88
Q30 62.02
Q31 58.81
Q32 45.18
Q33 8.22
Q34 26.43
Q35 61.51
Q36 34.65
Q37 34.02
Q38 34.88
Q39 45.45
Q40 26.21
Q41 50.72
Q42 7.97
Q43 45.18
Q44 26.38
Q45 58.81
Q46 26.22
Q47 58.7
Q48 13.54
Q49 45.64
Q50 26.25
Q51 62.19
Q52 34.8
Q53 50.76
Q54 13.45
Q55 50.53
Q56 26.81
Q57 8.4
Q58 62.14
Q59 50.85
Q60 13.64
Q61 34.46
Q62 25.91
Q63 45.92
Q64 34.51
Q65 62.24
Q66 13.43
Q67 62.03
Q68 34.46
Q69 58.72
Q70 58.8
Q71 61.91
Q72 34.74
Q73 26.46
Q74 26.39
Q75 45.66
Q76 34.56
Q77 45.51
Q78 34.37
Q79 34.55
Q80 61.23
Q81 45.83
Q82 8.07
Q83 62.0
Q84 34.02
Q85 34.28
Q86 50.78
Q87 25.84
Q88 50.73
Q89 61.96
Q90 45.15
Q91 62.0
Q92 45.84
Q93 34.38
Q94 13.72
Q95 46.09
Q96 61.38
Q97 34.7
Q98 50.72
Q99 62.12
Q100 59.09
Q101 45.55
Q102 50.98
Q103 

In [158]:
new_query={'name':'Franco','city':'Trento','job':'Producer'}

In [160]:
queries=pd.read_csv("./csv/queries.csv")
compute_query_score(new_query,um,queries,usersID,400)

U1       67.00
U2       88.23
U3        1.00
U4       78.00
U5       21.00
         ...  
U1996    86.95
U1997    86.91
U1998    87.49
U1999    13.16
U2000    63.21
Name: Q2000, Length: 2000, dtype: float64